# Introdução:

Análise de Dataset: Letras de Músicas

Este notebook compõe uma série de notebooks que tem como objetivo explorar e aplicar técnicas de NLP sobre o dataset `song_lyrics.csv`, conforme solicitado na prova prática para o CAEd.


# Importante

Para a tarefa (c) é necessário alterar o ambiente de execução para GPUs: T4.

# Configuração

In [ ]:
# Bibliotecas necessárias
import pandas as pd
import numpy as np
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Tarefa (c)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import Sequence
import random

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Baixando Dataset do Drive com gdown e extraindo zip
Link do Dataset: https://drive.google.com/file/d/13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c/view?usp=sharing

In [ ]:
# Baixar com gdown
!gdown 13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c

# Descompactar
with zipfile.ZipFile('dataset - letras.zip', 'r') as zip_ref:
    zip_ref.extractall('letras_dataset')

# Verificar arquivos
!ls letras_dataset

Downloading...
From (original): https://drive.google.com/uc?id=13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c
From (redirected): https://drive.google.com/uc?id=13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c&confirm=t&uuid=28400186-a8ea-4658-a7a2-28a4a1e49364
To: /content/dataset - letras.zip
100% 3.26G/3.26G [00:39<00:00, 81.9MB/s]
song_lyrics.csv


# Leituras

Caso a `amostra_song_lyrics10.csv` já exista e esteja incluída no ambiente. Basta pular para a etapa de pré-processamento. A etapa já carrega a base de forma processada, evitando passos extras.

## Opção 1: Amostra criada anteriormente

In [ ]:
# Deve-se subir a amostra para o ambiente do colab. O nome do arquivo deve estar na variável path entre aspas
path = 'amostra_song_lyrics10.csv'
df = pd.read_csv(path)
df.head()

,title,tag,artist,year,views,features,lyrics,language
0,Ducky,rap,North Star (USA),2004,143,"{""Meko the Pharaoh"",Christbearer}","""Oh you'll love her"" (repeats all throughout)\...",en
1,The Heist Revisited,rap,Big L,2000,5204,{},"[Big L]\nYeah, yeah\nYeah-yeah, yeah-yeah-yeah...",en
2,Chinese New Year,rap,Clipse,2006,22502,"{""Roscoe P. Coldchain""}",[Chorus: Pusha T & Pharrell]\nI'm at your door...,en
3,Pop That Cannon,rap,Cassidy,2004,677,"{""Styles P"",""Swizz Beatz""}","[Intro - Swizz Beatz]\nAooow! banger, let's go...",en
4,Been This Way,rap,Scribe,2003,1014,{},[Intro]\nEvery MC in this industry wants one t...,en


## Opção 2: Lendo com chuncksize e criando amostra

In [ ]:
# Caminho do CSV
csv_path = 'letras_dataset/song_lyrics.csv'

# Colunas desejadas
usecols = ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'language']

# Parâmetros
chunksize = 50_000  # número de linhas por chunk
sample_frac = 0.1   # fração da amostra por chunk (10%)

# Lista para armazenar amostras
samples = []

# Leitura por chunks
for chunk in pd.read_csv(csv_path, usecols=usecols, chunksize=chunksize):
  sample = chunk.sample(frac=sample_frac, random_state=42)
  samples.append(sample)

# Concatenar todos os pedaços amostrados
df = pd.concat(samples).reset_index(drop=True)

# Visualizar forma e colunas
print(f"Total de linhas na amostra: {df.shape[0]}")
df.head()

# Salva a amostra reduzida
df.to_csv('amostra_song_lyrics10.csv', index=False)
print("Arquivo salvo como amostra_song_lyrics10.csv")

Total de linhas na amostra: 513486
Arquivo salvo como amostra_song_lyrics.csv


# Pré-processamento

## Remoção de Livros e Poemas

In [ ]:
# Configurações
path = 'amostra_song_lyrics10.csv'
chunksize = 10000
limite_palavras_livro = 2000

# Contadores
total_entradas = 0
removidos_livros = 0
removidos_poemas = 0
exemplos_livros = []
exemplos_poemas = []
entradas_mantidas = []

# Função para detectar poemas
def analisar_estrutura(letra):
  if not isinstance(letra, str) or not letra.strip():
    return False

  linhas = [linha.strip() for linha in letra.split('\n') if linha.strip()]
  if len(linhas) < 8:
    return False

  total_linhas = len(linhas)
  palavras = ' '.join(linhas).split()
  diversidade = len(set(palavras)) / len(palavras) if palavras else 0
  media_palavras = len(palavras) / total_linhas
  linhas_repetidas = total_linhas - len(set(linhas))
  proporcao_repetidas = linhas_repetidas / total_linhas
  linhas_curtas = [linha for linha in linhas if len(linha.split()) < 7]
  proporcao_curtas = len(linhas_curtas) / total_linhas

  # Filtros básicos
  if media_palavras < 4 or diversidade < 0.3:
      return False

  # Score poético
  score_poetico = proporcao_curtas * (1 - proporcao_repetidas)

  return score_poetico >= 0.6

# Leitura por partes
for chunk in pd.read_csv(path, usecols=['title', 'tag', 'artist', 'year', 'views', 'lyrics', 'language'], chunksize=chunksize):
  chunk.dropna(subset=['lyrics'], inplace=True)
  total_entradas += len(chunk)

  # Detectar livros
  chunk['len_lyrics'] = chunk['lyrics'].str.split().apply(len)
  filtro_livros = chunk['len_lyrics'] > limite_palavras_livro
  livros = chunk[filtro_livros]
  if len(exemplos_livros) < 2:
    exemplos_livros.extend(livros[['title', 'artist', 'len_lyrics', 'lyrics']].head(2).to_dict('records'))
  removidos_livros += len(livros)

  # Restante
  chunk = chunk[~filtro_livros]

  # Detectar poemas nas tags relevantes
  chunk.loc[:, 'tag'] = chunk['tag'].astype(str).str.lower()
  filtro_tag_poema = chunk['tag'].isin(['misc', 'other'])

  poemas_idx = []
  for idx, row in chunk[filtro_tag_poema].iterrows():
    if analisar_estrutura(row['lyrics']):
      poemas_idx.append(idx)
      if len(exemplos_poemas) < 2:
        exemplos_poemas.append({
          'title': row['title'],
          'artist': row['artist'],
          'score_poetico': '≈ estrutural',
          'lyrics': row['lyrics']
        })

  removidos_poemas += len(poemas_idx)
  chunk = chunk.drop(poemas_idx)

  # (opcional) salvar para uso posterior
  entradas_mantidas.append(chunk)

# Concatenar final se desejar
df = pd.concat(entradas_mantidas, ignore_index=True)

# Relatório
print("Relatório de Limpeza:")
print(f"Total de entradas analisadas: {total_entradas}")
print(f"Entradas removidas (livros): {removidos_livros}")
print(f"Entradas removidas (poemas): {removidos_poemas}")
print(f"Total removido: {removidos_livros + removidos_poemas}")
print(f"Total final mantido: {len(df)}")

print("\nExemplos removidos (livros):")
for e in exemplos_livros:
  print(f" - {e['title']} ({e['artist']}) | {e['len_lyrics']} palavras")
  print(f"Trecho:\n{e['lyrics'][:300]}\n")

print("\nExemplos removidos (poemas):")
for e in exemplos_poemas:
  print(f" - {e['title']} ({e['artist']})")
  print(f"Trecho:\n{e['lyrics'][:300]}\n")

Relatório de Limpeza:
Total de entradas analisadas: 513486
Entradas removidas (livros): 2488
Entradas removidas (poemas): 2960
Total removido: 5448
Total final mantido: 508038

Exemplos removidos (livros):
 - Paradise Lost Book 9 (John Milton) | 9030 palavras
Trecho:
No more of talk where God or Angel guest
With Man, as with his friend, familiar us'd
To sit indulgent, and with him partake
Rural repast; permitting him the while
Venial discourse unblam'd. I now must change
Those notes to tragick; distrust, and breach
Disloyal on the part of Man, revolt
And disobed

 - Rappers Delight (Sugarhill Gang) | 3195 palavras
Trecho:
[Chorus: Wonder Mike]
I said a hip-hop, the hippie, the hippie
To the hip, hip-hop and you don't stop the rockin'
To the bang-bang boogie, say up jump the boogie
To the rhythm of the boogie, the beat

[Verse 1: Wonder Mike]
Now, what you hear is not a test, I'm rapping to the beat
And me, the groove


Exemplos removidos (poemas):
 - The Tyger (William Blake)
Trecho:
T

## Limpeza no campo lyrics

In [ ]:
stop_words = set(stopwords.words('english'))

def clean_lyrics(text):
  text = text.lower()
  text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # remove URLs
  text = re.sub(r'\n+', ' ', text)
  text = re.sub(r'[^a-zA-Z ]', '', text)       # remove pontuação
  text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)  # Remove letras únicas isoladas
  text = re.sub(r'\s+', ' ', text).strip()     # Remove espaços múltiplos e bordas

  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
  return ' '.join(tokens)

df = df[df['language'] == 'en'].copy()
print(f"Total de letras em inglês: {df.shape[0]}")
df['lyrics'] = df['lyrics'].apply(clean_lyrics)
df.head()

Total de letras em inglês: 332731


,title,tag,artist,year,views,lyrics,language,len_lyrics
0,Ducky,rap,North Star (USA),2004,143,oh youll love repeats throughout intro christ ...,en,465
1,The Heist Revisited,rap,Big L,2000,5204,big yeah yeah yeahyeah yeahyeahyeahyeah yeah y...,en,696
2,Chinese New Year,rap,Clipse,2006,22502,chorus pusha pharrell im door eyes like judgin...,en,691
3,Pop That Cannon,rap,Cassidy,2004,677,intro swizz beatz aooow banger lets go swizz m...,en,562
4,Been This Way,rap,Scribe,2003,1014,intro every mc industry wants one thing best t...,en,785


# Tarefa (c): Geração de Letras

Objetivo: gerar texto musical a partir de modelos sequenciais

## Pré-processamento do Corpus

In [ ]:
# Agrupar as letras num único texto longo e converter para vetor de tokens/caracteres.
# Juntar todas as letras em um único texto
text = " ".join(df['lyrics'].values).lower()

# Mapeamento caractere <-> índice
chars = sorted(list(set(text)))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for i, c in enumerate(chars)}

# Converter texto inteiro em sequência de índices
seq = [char2idx[c] for c in text]

## Criar Dados de Treino para o LSTM

In [ ]:
# Parâmetros do modelo
seq_length = 60
batch_size = 256
vocab_size = len(chars)
total_sequences = len(seq) - seq_length

# Gerador de batches para treino
class LyricsGenerator(Sequence):
  def __init__(self, data, seq_length, batch_size, vocab_size):
    self.data = data
    self.seq_length = seq_length
    self.batch_size = batch_size
    self.vocab_size = vocab_size
    self.indices = np.arange(0, len(data) - seq_length, 5)

  def __len__(self):
    return len(self.indices) // self.batch_size

  def __getitem__(self, idx):
    batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]
    X = np.zeros((self.batch_size, self.seq_length, 1))
    y = np.zeros((self.batch_size, self.vocab_size))

    for i, start_idx in enumerate(batch_indices):
      seq_x = self.data[start_idx : start_idx + self.seq_length]
      seq_y = self.data[start_idx + self.seq_length]
      X[i] = np.reshape(seq_x, (self.seq_length, 1)) / float(self.vocab_size)
      y[i][seq_y] = 1.0

    return X, y

train_generator = LyricsGenerator(seq, seq_length, batch_size, vocab_size)

## Criar o Modelo LSTM

In [ ]:
# Modelo LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(seq_length, 1)))
model.add(LSTM(128))
model.add(Dropout(0.3))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

## Treinamento e Avaliação

In [ ]:
# Treinar
checkpoint = keras.callbacks.ModelCheckpoint("lyrics_model.h5", save_best_only=True)
model.fit(train_generator, epochs=100, steps_per_epoch=3000, callbacks=[checkpoint])

Epoch 1/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 49s 16ms/step - loss: 2.7766
Epoch 2/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 2.3348
Epoch 3/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 2.1666
Epoch 4/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 2.0838
Epoch 5/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 2.0248
Epoch 6/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.9903
Epoch 7/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.9624
Epoch 8/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.9270
Epoch 9/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.9021
Epoch 10/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.8911
Epoch 11/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.8765
Epoch 12/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.8643
Epoch 13/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.8554
Epoch 14/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - 

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6722
Epoch 84/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 47s 16ms/step - loss: 1.6746
Epoch 85/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 47s 16ms/step - loss: 1.6692
Epoch 86/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 48s 16ms/step - loss: 1.6651
Epoch 87/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 47s 16ms/step - loss: 1.6668
Epoch 88/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6535
Epoch 89/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6589
Epoch 90/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6625
Epoch 91/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6622
Epoch 92/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6580
Epoch 93/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6677
Epoch 94/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss: 1.6630
Epoch 95/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - loss: 1.6523
Epoch 96/100
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - loss

In [ ]:
def sample(preds, temperature=0.8):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

# Função para gerar letra
def generate_lyrics(seed_text, length=300, temperature=0.8):
    seed_text = seed_text.lower()
    pattern = [char2idx[c] for c in seed_text if c in char2idx]
    output = seed_text

    for _ in range(length):
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(vocab_size)
        prediction = model.predict(x, verbose=0)
        index = sample(prediction[0], temperature)
        result = idx2char[index]
        output += result
        pattern.append(index)
        pattern = pattern[1:]

    return output

# Exemplo
seed = "just stay with"
print(generate_lyrics(seed, length=100, temperature=0.9))

just stay without something calming money baby bitch line demasse shadows em means bep four come parevl could ei m
